In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [4]:
spark

Showing Age as a string so we are adding inferSchema

In [14]:
df_pyspark=spark.read.csv('aa.csv',header=True,inferSchema=True)

In [15]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [16]:
df_pyspark.show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  dfsds|  23|         3| 35000|
|   dsfe|  30|         4| 40000|
|   fsff|null|         1| 25000|
|    fvf|  27|         3|  null|
|cvcvvdv|null|      null|400000|
|  ssddd|null|         6| 30000|
+-------+----+----------+------+



In [17]:
#drop columns
df_pyspark.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  23|         3| 35000|
|  30|         4| 40000|
|null|         1| 25000|
|  27|         3|  null|
|null|      null|400000|
|null|         6| 30000|
+----+----------+------+



In [19]:
#drop all row null values presented
df_pyspark.na.drop().show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|dfsds| 23|         3| 35000|
| dsfe| 30|         4| 40000|
+-----+---+----------+------+



In [23]:
#Any==how, any executes entire row if contains any null values(by default it is any)
#'all' executes the columns contain null entire column.
df_pyspark.na.drop(how="all").show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  dfsds|  23|         3| 35000|
|   dsfe|  30|         4| 40000|
|   fsff|null|         1| 25000|
|    fvf|  27|         3|  null|
|cvcvvdv|null|      null|400000|
|  ssddd|null|         6| 30000|
+-------+----+----------+------+



In [30]:
#Threshold(it keeps at least 2 non-null values are present otherwise that rows will deleted, 
#thresh can be any value)
df_pyspark.na.drop(how='any',thresh=2).show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  dfsds|  23|         3| 35000|
|   dsfe|  30|         4| 40000|
|   fsff|null|         1| 25000|
|    fvf|  27|         3|  null|
|cvcvvdv|null|      null|400000|
|  ssddd|null|         6| 30000|
+-------+----+----------+------+



In [32]:
#subset--all record will deleted if there are any null values in the specified column
df_pyspark.na.drop(how="any",subset=["Experience"]).show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|dfsds|  23|         3| 35000|
| dsfe|  30|         4| 40000|
| fsff|null|         1| 25000|
|  fvf|  27|         3|  null|
|ssddd|null|         6| 30000|
+-----+----+----------+------+



In [42]:
#filling the Missing value
df_pyspark.na.fill(3,['Experience','Age']).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  dfsds| 23|         3| 35000|
|   dsfe| 30|         4| 40000|
|   fsff|  3|         1| 25000|
|    fvf| 27|         3|  null|
|cvcvvdv|  3|         3|400000|
|  ssddd|  3|         6| 30000|
+-------+---+----------+------+



In [45]:
from pyspark.ml.feature import Imputer
imputer= Imputer(
    inputCols=['Age','Experience','Salary'], 
    outputCols=["{}_imputed]".format(c) for c in ['Age','Experience','Salary']]
    ).setStrategy("mean")

In [46]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-------+----+----------+------+------------+-------------------+---------------+
|   Name| Age|Experience|Salary|Age_imputed]|Experience_imputed]|Salary_imputed]|
+-------+----+----------+------+------------+-------------------+---------------+
|  dfsds|  23|         3| 35000|          23|                  3|          35000|
|   dsfe|  30|         4| 40000|          30|                  4|          40000|
|   fsff|null|         1| 25000|          26|                  1|          25000|
|    fvf|  27|         3|  null|          27|                  3|         106000|
|cvcvvdv|null|      null|400000|          26|                  3|         400000|
|  ssddd|null|         6| 30000|          26|                  6|          30000|
+-------+----+----------+------+------------+-------------------+---------------+

